In [2]:
%pip install -q ipywidgets
%pip install -q numpy
%pip install -q pandas
%pip install -q matplotlib

from one_locus_two_alleles_app import OneLocusSimApp
OneLocusSimApp().run_application()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
